In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np

# Просмотр

In [60]:
data = pd.read_excel('data/initial.xlsx')

Посмотрим максимальную и минимальную длины сообщений

In [52]:
max_len = 0
min_len = 100000
for i in data.index:
    if type(data['message'][i]) == str:
        max_len = max(max_len, len(data['message'][i]))
        min_len = min(min_len, len(data['message'][i]))
print(max_len, min_len)

943 1


# Исправление орфографии и пунктуации

In [ ]:
# Импортируем библиотеки
from transformers import AutoModelForSeq2SeqLM, T5TokenizerFast

# Зададим название выбронной модели из хаба
MODEL_NAME = 'UrukHan/t5-russian-spell'
MAX_INPUT = 256

# Загрузка модели и токенизатора
tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

In [87]:
input_sequences = ['сеглдыя хорош ден']   # или можно использовать одиночные фразы:  input_sequences = 'сеглдыя хорош ден'

task_prefix = "Spell correct: "
if type(input_sequences) != list: input_sequences = [input_sequences]
encoded = tokenizer(
    [task_prefix + sequence for sequence in input_sequences],
    padding="longest",
    max_length=943,
    truncation=True,
    return_tensors="pt",
)['input_ids']

predicts = model.generate(encoded)

tokenizer.batch_decode(predicts, skip_special_tokens=True)

/home/jupyter/.local/lib/python3.8/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (256) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['Сегодня хорош день.']

In [80]:
data_grammar_fix = data.copy(deep=True)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

for i in tqdm(data.index[2000:]):
    if data['is_visitor_message'][i] == 'ОПЕРАТОР' and type(data['message'][i]) == str and data['message'][i][0] != '!' and ('севстар' not in data['message'][i].lower()):
        input_sequences = [data['message'][i]]
        
        encoded = tokenizer(
            [task_prefix + sequence for sequence in input_sequences],
            padding="longest",
            max_length=MAX_INPUT,
            truncation=True,
            return_tensors="pt",
        )['input_ids']

        predicts = model.generate(encoded)    # # Прогнозирование

        output = tokenizer.batch_decode(predicts, skip_special_tokens=True)
        data_grammar_fix.loc[i, 'message'] = output[0]
    if i % 1000 == 0:
        data_grammar_fix.to_csv('data/grammar_fix.csv')
        print(i, "samples done!")

  0%|          | 1/21360 [00:00<41:31,  8.57it/s]

2000 samples done!


  5%|▍         | 1002/21360 [02:51<42:52,  7.92it/s] 

3000 samples done!


  9%|▉         | 2001/21360 [05:39<59:49,  5.39it/s]  

...

 94%|█████████▎| 20001/21360 [58:13<04:24,  5.13it/s]

22000 samples done!


 98%|█████████▊| 21001/21360 [1:01:16<01:53,  3.15it/s]

23000 samples done!


100%|██████████| 21360/21360 [1:02:12<00:00,  5.72it/s]


In [98]:
data_grammar_fix.to_csv('data/grammar_fix.csv')

# Удаление улиц

In [15]:
df = pd.read_csv('data/grammar_fix.csv')

In [3]:
streets_df = pd.read_csv('/home/jupyter/datasphere/project/datasets/sevstar_data/cities_list.csv')

In [4]:
streets = list(streets_df['name'])
short_names_preview = list(set(streets_df['prefix']))

In [5]:
from numpy import nan
short_names = [item for item in short_names_preview if not(pd.isnull(item)) == True]

In [ ]:
short_names.append('д')
short_names.append('д.')
short_names.append('Д')
short_names.append('Д.')
short_names.append('кв.')
short_names.append('КВ.')
short_names.append('кв')
short_names.append('кВ')
short_names.append('Кв')
short_names.append('Кв.')
short_names.append('кВ.')
short_names.append('Ул.')
short_names.sort()

short_names_fixed = []
for word in short_names:
    short_names_fixed.append(' ' + word + ' ')
short_names_fixed

In [8]:
def del_streets(string):
    for name in streets:
        string = string.replace(name, '<street>')

    for short_name in short_names_fixed:
        string = string.replace(short_name, ' ')

    return string

In [16]:
changed = 0
for i in tqdm(df.index):
    if df['is_visitor_message'][i] == 'АБОНЕНТ':
        prev = df.loc[i, 'message']
        df.loc[i, 'message'] = del_streets(df['message'][i].lower())
        if prev != df.loc[i, 'message']:
            changed += 1
df.to_csv('data/grammar_cities_fix.csv')

100%|██████████| 23360/23360 [00:04<00:00, 5702.52it/s]


In [17]:
changed

5689

# Генерация триплетов

В качестве негативной пары будем брать случайный ответ пользователя

Используем окно контекста длины 6

In [ ]:
df = pd.read_csv('data/grammar_cities_fix.csv')
df.head()

In [19]:
grouped_df = df

In [ ]:
df_no_operator = grouped_df[grouped_df['is_visitor_message'] != 'ОПЕРАТОР']
df_no_operator.head()

In [ ]:
df_operator = grouped_df[grouped_df['is_visitor_message'] == 'ОПЕРАТОР']
df_operator.head()

In [23]:
operator_idx = np.array(list(df_operator.index))

In [24]:
def get_random(seq, df, exc=None):
    if len(seq) == 1:
        return seq[0]
                                      
    selected = None
    while selected is None or df['message'][selected] == df['message'][exc] or (df['message'][selected] in df['message'][exc]) or (df['message'][exc] in df['message'][selected]):
        selected = np.random.choice(seq)
    return selected

In [2]:
import pickle

def write_list(filename, to_store):
    with open(filename, 'wb') as fp:
        pickle.dump(to_store, fp)

def read_list(filename):
    with open(filename, 'rb') as fp:
        n_list = pickle.load(fp)
        return n_list

In [26]:
anchors = []
pos = []
neg = []

context = ['', '', '', '', '', '']
cur_id = None

for i in tqdm(df.index):
    if df['is_visitor_message'][i] != 'ОПЕРАТОР':
        if cur_id == df['chat_id'][i]:
            context.pop(0)
            context.append(df['message'][i])
        else:
            context = ['', '', '', '', '', df['message'][i]]
            cur_id = df['chat_id'][i]
            
        if i+1 < df.shape[0] and df['is_visitor_message'][i+1] == 'ОПЕРАТОР':
            write_list(f'anchors/{i}.pickle', context)
            anchors.append(i)
            neg_id = get_random(operator_idx, df, i+1)
            neg.append(grouped_df['message'][neg_id])
            pos.append(grouped_df['message'][i+1])

100%|██████████| 23360/23360 [00:07<00:00, 3115.96it/s]


In [27]:
d = {'anchor' : anchors,
    'positive' : pos,
    'negative' : neg}
    
df = pd.DataFrame(data=d)
df.head()

,anchor,positive,negative
0,0,!ЗАПРОС_В_ЕРП,"Здравствуйте! Уточните, пожалуйста, адрес, либ..."
1,2,Здравствуйте.,"Да, всё верно."
2,6,!ЗАПРОС_В_ЕРП,Адреса корреспонденции доступны у нас на сайте
3,8,Заявка оформлена.,Удаленный тест не выявил видимых неполадок под...
4,11,"Здравствуйте. Скажите, пожалуйста, Ваш адрес и...","Отключите, пожалуйста, роутер из розетки на 20..."


In [28]:
df.to_csv('data/triplets_cities_with_context.csv')

In [29]:
df.shape

(8723, 3)

# Конкатенация контекста

In [2]:
import pandas as pd
import re
import pickle

def read_list(filename):
    with open(filename, 'rb') as fp:
        n_list = pickle.load(fp)
        return n_list

df = pd.read_csv('data/triplets_cities_with_context_scores.csv')
anchors = df['anchor']
anchors_text = []
mn, mx = 10000000, 0
for i in anchors:
    cur = read_list(f'anchors/{i}.pickle')
    not_empty = 0
    for j in range(len(cur)):
        if len(cur[j]) != 0:
            break
        not_empty+=1
    cur = ' '.join(cur[not_empty:])
    anchors_text.append(cur)
    count = len(re.findall(r'\w+', cur))
    mx = max(count, mx)
    mn = min(count, mn)
print(mn, mx)

0 159


Т.к. токенайзер имеет max_length=2048, можно сложить окно контекста в одну строку.

In [ ]:
df['anchor_one_str'] = anchors_text
df.head()

In [5]:
df = df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
df.to_csv('data/triplets_cities_context_onestr_scores.csv')

# Предподсчет скора оценщика

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel

class RewardModel(nn.Module):
        def __init__(self, model_name):
            super(RewardModel, self).__init__()
            self.checkpoint = model_name
            self.bert = AutoModel.from_pretrained(model_name,
                                                  return_dict=False)
            self.layer_norm = nn.LayerNorm(768)
            self.dropout = nn.Dropout(0.3)
            self.dense = nn.Sequential(
                nn.Linear(768, 512),
                nn.LeakyReLU(negative_slope=0.01),
                nn.Dropout(0.3),
                nn.Linear(512, 1),
                nn.Sigmoid()
            )

        def forward(self, input_ids, token_type_ids, attention_mask):
            
            model_output = self.bert(input_ids=input_ids, 
                                     token_type_ids = token_type_ids,
                                     attention_mask=attention_mask)
            
            last_hidden_states = model_output[0]
            pooled_output = last_hidden_states[:,0]
            pooled_output = self.layer_norm(pooled_output)
            pooled_output = self.dropout(pooled_output)
            preds = self.dense(pooled_output)
            return preds


#Create model object and init pretrain weights:
reward_name = "ai-forever/ruBert-base"
tokenizer_reward=AutoTokenizer.from_pretrained(reward_name)
model_reward = RewardModel(reward_name)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
if device == 'cpu':
    model_reward.load_state_dict(torch.load('./ruBert-base-reward/pytorch_model.bin', map_location=torch.device('cpu')), strict=False)
else:
    model_reward.load_state_dict(torch.load('./ruBert-base-reward/pytorch_model.bin'), strict=False)
model_reward.to(device);

In [2]:
def get_score(anchor, positive):
    with torch.no_grad():
        encoded_input = tokenizer_reward(anchor, positive,
                                          truncation=True,
                                          add_special_tokens=True,
                                          max_length=512,
                                          padding='max_length',
                                          return_tensors='pt')

        encoded_input = encoded_input.to(device)
        r_score = model_reward(**encoded_input).cpu().flatten().numpy()[0]
    return r_score

In [ ]:
import pandas as pd
df = pd.read_csv('data/triplets_cities_context_onestr_scores.csv')
df.head()

In [5]:
import pickle

def write_list(filename, to_store):
    with open(filename, 'wb') as fp:
        pickle.dump(to_store, fp)

In [6]:
from tqdm import tqdm
scores = []

for i in tqdm(df.index):
    context = df["anchor_one_str"][i]
    scores.append(get_score(context, df['positive'][i]))
    if i % 500 == 0:
        write_list('data/scores.pckl', scores)
        print(f'Scores for {i} samples saved')

  0%|          | 1/8723 [00:00<1:14:15,  1.96it/s]

Scores for 0 samples saved


  6%|▌         | 501/8723 [04:04<1:06:51,  2.05it/s]

Scores for 500 samples saved


 11%|█▏        | 1001/8723 [08:06<1:01:53,  2.08it/s]

...

 92%|█████████▏| 8001/8723 [1:04:51<05:53,  2.04it/s]

Scores for 8000 samples saved


 97%|█████████▋| 8501/8723 [1:08:54<01:50,  2.02it/s]

Scores for 8500 samples saved


100%|██████████| 8723/8723 [1:10:43<00:00,  2.06it/s]


In [7]:
write_list('data/scores.pckl', scores)

In [8]:
df['score'] = scores
df.to_csv('data/triplets_cities_context_onestr_scores.csv')